In [1]:
#Imprting packages
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose 

In [2]:
#Function which takes a text file as parameter and convert the text into audio,when the function is called.
engine = pyttsx3.init()
def text2speech(text_file):
    pose=text_file
    f=open(pose,'r')
    text=f.read()
    f.close()
    engine.setProperty('rate',150)
    engine.say(text)
    engine.runAndWait()

In [3]:
#Function to calculate angles between two bones which will be used further.
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [4]:
cap = cv2.VideoCapture(0)

#Creating lable variable
label="Unknown Pose"

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        ##STEP I 
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_shoulder= [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # STEP II:Calculate angle between the joints
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle=calculate_angle(right_shoulder,right_elbow,right_wrist)
            left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            left_hip_angle=calculate_angle(left_knee,left_hip,left_shoulder)
            right_hip_angle=calculate_angle(right_knee,right_hip,right_shoulder)
            left_knee_angle=calculate_angle(left_ankle,left_knee,left_hip)
            right_knee_angle=calculate_angle(right_ankle,right_knee,right_hip)
           
            #STEP III: Classifing poses
            #Check if hands are staright or not
            if left_elbow_angle > 165 and left_elbow_angle < 195 and right_elbow_angle > 165 and right_elbow_angle < 195:
                if left_shoulder_angle > 80 and left_shoulder_angle < 110 and right_shoulder_angle > 80 and right_shoulder_angle < 110:
                    #Warrior II pose
                    if left_knee_angle > 165 and left_knee_angle < 195 or right_knee_angle > 165 and right_knee_angle < 195:
                        if left_knee_angle > 90 and left_knee_angle < 120 or right_knee_angle > 90 and right_knee_angle < 120:
                            label="warrior II Pose"
                            text2speech("Warrior_II.txt")
                    #T Pose
                    if left_knee_angle > 160 and left_knee_angle < 195 and right_knee_angle > 160 and right_knee_angle < 195:
                        label = 'T Pose'    
                        text2speech("TPose.txt")
            #Tree Pose
            if left_knee_angle > 165 and left_knee_angle < 195 or right_knee_angle > 165 and right_knee_angle < 195 :
                if left_knee_angle > 315 and left_knee_angle < 335 or right_knee_angle > 25 and right_knee_angle < 45 :
                    label = 'Tree Pose'
                    text2speech("treepose.txt")
                    
            else:
                label = 'Unknown Pose'

            print("Right elbow angle is:",right_elbow_angle)
            print("Left elbow angle is:",left_elbow_angle)
            print("Right  ")
        
        except:
            pass
        
        #Showing label on the output screen
        cv2.putText(image, label,(10,60),cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0), 1, cv2.LINE_AA)

        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2))
                
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()

Right elbow angle is: 175.12763790231028
Left elbow angle is: 143.50198454590233
Right  
Right elbow angle is: 75.02808451546473
Left elbow angle is: 16.724052688231904
Right  
Right elbow angle is: 28.71021479933898
Left elbow angle is: 12.480625628714803
Right  
Right elbow angle is: 12.37203222426371
Left elbow angle is: 9.95228253338661
Right  
Right elbow angle is: 22.612334332387327
Left elbow angle is: 4.163960269716182
Right  
Right elbow angle is: 43.767290121591074
Left elbow angle is: 8.4868654609145
Right  
Right elbow angle is: 95.60597128050723
Left elbow angle is: 123.9751811977971
Right  
Right elbow angle is: 161.86150490367646
Left elbow angle is: 151.99775295575884
Right  
Right elbow angle is: 165.45006840294334
Left elbow angle is: 146.53475062171955
Right  
Right elbow angle is: 161.46372510687672
Left elbow angle is: 48.508178397568415
Right  
Right elbow angle is: 137.62504858344033
Left elbow angle is: 15.75738829037696
Right  
Right elbow angle is: 27.11113115